In [187]:
#pip install --upgrade ibm-watson

In [175]:
import pandas as pd
import json
from translate import Translator                    # translation api 
#pd.set_option('display.max_colwidth', -1)



#analyser = SentimentIntensityAnalyzer()                     # library documention https://github.com/cjhutto/vaderSentiment/blob/master/README.rst
translator = Translator(to_lang = 'english')

                         # opening one json for testing 
df = pd.read_json (r'C:\Users\20193673\Desktop\Quartile 4\!DataChallenge\airlines-1558527599826.json', lines=True)
pd.set_option('display.max_columns', None)

In [179]:
import watson_developer_cloud as WDC
from watson_developer_cloud.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions, SentimentOptions, CategoriesOptions
natural_language_understanding = WDC.NaturalLanguageUnderstandingV1(                                         
    version='2018-11-16',
    iam_apikey='KdkxhBFqePh5Fwfhzch-qaMNsnOj0gboa6Svx61pEdSa', # Use your API key here
    url= 'https://api.eu-de.natural-language-understanding.watson.cloud.ibm.com/instances/565d3a29-ea2e-4146-a8aa-7847c474feaf')


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: watson-developer-cloud moved to ibm-watson. To get updates, use the new package.
  


In [182]:
def sentiment_analyzer_scores(text, engl=True):
    if engl:                                     # if the leng is engl it skips the translation
        trans = text
    else:
        trans = translator.translate(text).text # it translates the text
       
    def Sentiment_score(input_text): 
    # Input text can be sentence, paragraph or document
        response = natural_language_understanding.analyze (text = input_text, features = Features(sentiment=SentimentOptions()), language='en').get_result()
        res = response.get('sentiment').get('document').get('score')
        return res    
    
    score =  Sentiment_score(trans)  # gets the scores from the library 
    
    
    lb = score
    if lb >= 0.33:                              # divides the sentiment score into (-1,0,1) 
        return 1                                # can be changed to even more catagories
    elif (lb > -0.33) and (lb < 0.33):               
        return 0
    else:
        return -1

In [183]:
sentiment = []                        # gets a list of all the sentimental values of the tweets
for x in range(len(df)) :  
    
    sentiment.append(sentiment_analyzer_scores(str(df['text'][x])))

In [184]:
df.insert(28 ,'sentiment', sentiment)   # inserting the sentimental values into the dataframe

In [186]:
df[['text','sentiment']].sample(10)

,text,sentiment
7438,RT @SalmaSlili: Jay park a fait un live avec u...,-1
13202,RT @Ryanair: Our friends @Laudamotion1 have ma...,1
9984,@itastepaint This isn't how we wanted things t...,-1
3241,RT @ihoelt: Diese Europa-Wahl fühlt sich ander...,0
4863,RT @Ryanair: Our friends @Laudamotion1 have ma...,1
13560,@British_Airways,0
12875,@MaMaDolson @British_Airways Enjoy,1
4263,RT @jblefevre60: This #digital screen synchron...,-1
5348,RT @francediplo_ES: #Ecuador 🇪🇨\n@JBLemoyne re...,0
4056,@VirginAtlantic can you advise me on taking pr...,0
